# **Modeling and Evaluation Notebook**

## Objectives

- Answer Business Requirement 2: Develop a Machine Learning model to classify cherry leaves as Healthy or Infected, enabling the prediction of powdery mildew presence.

## Inputs

Dataset
- inputs/mildew_dataset/cherry-leaves/train
- inputs/mildew_dataset/cherry-leaves/validation
- inputs/mildew_dataset/cherry-leaves/test

Precomputed Features (from Data Visualization Notebook)
- Image Shape Standardization → 128x128x3 for consistency across models.
- Class Distribution Analysis → Ensures balanced dataset splits.
- Pixel Intensity Distribution → Confirms brightness variations relevant for classification.

## Outputs

### Data Processing & Visualization
- **Dataset Distribution Plot** → Confirms balanced data split across training, validation, and test sets.  
- **Data Augmentation Visualization** → Showcases applied transformations (rotation, flipping, zooming).  

### Model Training & Evaluation
- **Baseline CNN Models (Sigmoid & Softmax)** → Initial experiments to establish a benchmark.  
- **MobileNetV2 Fine-Tuning** → Explored optimized architectures with different hyperparameters.  
- **Best Model Selection** → Chose the most balanced model based on test accuracy, generalization, and robustness.  
- **Saved Trained Model** → Final MobileNetV2 model stored for deployment.  

### Model Performance & Explainability
- **Learning Curves** → Visualizes loss and accuracy trends over epochs.  
- **Histograms** → Displays predicted probability distributions.  
- **Overfitting & Generalization Check** → Assesses potential overfitting using accuracy and loss gaps.  
- **Confusion Matrices** → Shows classification performance for train, validation, and test sets.  
- **Classification Reports** → Provides precision, recall, and F1-score analysis.  
- **ROC Curves** → Evaluates model performance using Receiver Operating Characteristic analysis.  
- **Business Goal Validation** → Confirms if the model meets the required accuracy threshold.  

## Additional Comments

- **Business Impact:** Enables early detection of powdery mildew, reducing manual inspection and improving monitoring.  
- **Data-Driven Improvements:** Model refinements were based on data insights, ensuring balanced class distribution.  
- **Deployment:** The optimized model is ready for Streamlit integration for real-world use.  



---

# Set Data Directory

---

## Import Necessary Packages

In [ ]:
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from matplotlib.image import imread

## Set Working Directory

In [ ]:
cwd= os.getcwd()

In [ ]:
os.chdir("/workspaces/mildew-detector")
print("You set a new current directory")

#### Confirm the new current directory

In [ ]:
work_dir = os.getcwd()
work_dir

## Set Input Directories

In [ ]:
# Set input directories
my_data_dir = 'inputs/mildew_dataset/cherry-leaves'
train_path = os.path.join(my_data_dir, 'train')
val_path = os.path.join(my_data_dir, 'validation')
test_path = os.path.join(my_data_dir, 'test')

## Set Output Directory

In [ ]:
version = "v1"
file_path = f'outputs/{version}'

if 'outputs' in os.listdir(work_dir) and version in os.listdir(work_dir + '/outputs'):
    print('Old version is already available create a new version.')
    pass
else:
    os.makedirs(name=file_path)

## Set Label Names

In [ ]:
# Set the labels for the images
labels = os.listdir(train_path)
print('Label for the images are', labels)

## Set Image Shape

In [ ]:
## Import saved image shape embedding
import joblib
version = 'v1'
image_shape = joblib.load(filename=f"outputs/{version}/image_shape.pkl")
image_shape

---

## Number of Images in Train, Test and Validation Data

In [ ]:
import pandas as pd

# Create an empty dictionary
data = {
    'Set': [],
    'Label': [],
    'Frequency': []
}

# Define dataset folders
folders = ['train', 'validation', 'test']

# Loop through each dataset split and count images
for folder in folders:
    for label in labels:
        path = os.path.join(my_data_dir, folder, label)
        num_images = len(os.listdir(path)) if os.path.exists(path) else 0  
        data['Set'].append(folder)
        data['Label'].append(label)
        data['Frequency'].append(num_images)
        print(f" {folder}/{label}: {num_images} images")

# Convert dictionary to DataFrame
df_freq = pd.DataFrame(data)

### Bar Chart - Image Distribution

In [ ]:
sns.set_style("whitegrid")
plt.figure(figsize=(8, 5))
sns.barplot(data=df_freq, x='Set', y='Frequency', hue='Label')
plt.title("Image Distribution in Dataset")
plt.xlabel("Dataset Split")
plt.ylabel("Number of Images")
plt.savefig(f'{file_path}/labels_distribution.png', bbox_inches='tight', dpi=150)
plt.show()

---

# Implement Data Augmentation

---

### ImageDataGenerator

In [ ]:
# Import TensorFlow/Keras ImageDataGenerator
from tensorflow.keras.preprocessing.image import ImageDataGenerator

## Augment Training, Validation, and Test Sets

- Initialize ImageDataGenerator for Data Augmentation

In [ ]:
# Define Augmentation for Training Set
augmented_image_data = ImageDataGenerator(rotation_range=20,
                                          width_shift_range=0.10,
                                          height_shift_range=0.10,
                                          shear_range=0.1,
                                          zoom_range=0.1,
                                          horizontal_flip=True,
                                          vertical_flip=True,
                                          fill_mode='nearest',
                                          rescale=1./255
                                          )

- Augment Training Image Dataset

In [ ]:
batch_size = 16  # Set batch size
train_set = augmented_image_data.flow_from_directory(train_path,
                                                     target_size=image_shape[:2],
                                                     color_mode='rgb',
                                                     batch_size=batch_size,
                                                     class_mode='binary',
                                                     shuffle=True
                                                     )

train_set.class_indices

- Augment Validation Image Dataset

In [ ]:
validation_set = ImageDataGenerator(rescale=1./255).flow_from_directory(val_path,
                                                                        target_size=image_shape[:2],
                                                                        color_mode='rgb',
                                                                        batch_size=batch_size,
                                                                        class_mode='binary',
                                                                        shuffle=False
                                                                        )

validation_set.class_indices

- Augment Test Image Dataset

In [ ]:
test_set = ImageDataGenerator(rescale=1./255).flow_from_directory(test_path,
                                                                  target_size=image_shape[:2],
                                                                  color_mode='rgb',
                                                                  batch_size=batch_size,
                                                                  class_mode='binary',
                                                                  shuffle=False
                                                                  )

test_set.class_indices

---

## Visualization of Augmented Images

### Plot Augmented Training Image

In [ ]:
for _ in range(3):
    img, label = next(train_set)
    print(img.shape)  # (1,256,256,3)
    plt.imshow(img[0])
    plt.show()

### Plot Augmented Validation and Test Images

In [ ]:
for _ in range(3):
    img, label = next(validation_set)
    print(img.shape)  # (1,256,256,3)
    plt.imshow(img[0])
    plt.show()

In [ ]:
for _ in range(3):
    img, label = next(test_set)
    print(img.shape)  # (1,256,256,3)
    plt.imshow(img[0])
    plt.show()

### Save Class Indices

In [ ]:
joblib.dump(value=train_set.class_indices,
            filename=f"{file_path}/class_indices.pkl")

### Compare Multiple Augmented Images in a Grid

In [ ]:
def plot_augmented_images_grid(data_generator, num_images=10):
    """Displays a grid of augmented images to visualize transformation effects."""
    img_batch, label_batch = next(data_generator)

    fig, axes = plt.subplots(2, num_images // 2, figsize=(15, 6))
    
    for i in range(num_images):
        ax = axes[i // (num_images // 2), i % (num_images // 2)]
        ax.imshow(img_batch[i])
        ax.axis("off")

    plt.suptitle("Augmented Image Variations (Training Set)")
    plt.show()

# Display the augmented image grid
plot_augmented_images_grid(train_set)

---

# Model Creation

---

### Import Libraries

In [ ]:
import os
from tensorflow.keras.applications import MobileNetV2
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import GlobalAveragePooling2D, Dense, Dropout
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.regularizers import l2

## MobileNetV2

MobileNetV2 was chosen for its efficiency, speed, and strong feature extraction while maintaining high accuracy with fewer parameters. Its lightweight architecture makes it ideal for deployment in resource-constrained environments. Details are provided in the README.

In [ ]:
# Define hyperparameter
learning_rate = 0.00005
l2_lambda = 0.004
dropout_rate = 0.65
activation_function = "tanh"
patience_value = 4

# Create MobileNetV2 Model
model_mobilenet = Sequential(
    [
        MobileNetV2(weights="imagenet", include_top=False, input_shape=(128, 128, 3)),
        GlobalAveragePooling2D(),
        Dense(128, activation=activation_function, kernel_regularizer=l2(l2_lambda)),
        Dropout(dropout_rate),
        Dense(2, activation="softmax"),
    ]
)

# Compile Model
model_mobilenet.compile(
    optimizer=Adam(learning_rate=learning_rate),
    loss="sparse_categorical_crossentropy",
    metrics=["accuracy"],
)

### Model Summary

In [ ]:
# Print Model Summary
model_mobilenet.summary()

---

## Model Training

### Start Training Timer

In [ ]:
import time
import pandas as pd

# Start time tracking
start_time = time.time()

### Early Stopping Implementation

In [ ]:
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau, ModelCheckpoint

# Set EarlyStopping & Learning Rate Scheduler
early_stop = EarlyStopping(
    monitor="val_loss", mode="min", patience=patience_value, restore_best_weights=True
)

lr_scheduler = ReduceLROnPlateau(
    monitor="val_loss", mode="min", factor=0.5, patience=2, min_lr=1e-6
)

m_checkpoint = ModelCheckpoint(
    filepath="outputs/mobilenet.h5",
    monitor="val_accuracy",
    mode="max",
    save_best_only=True,
)

### Fit Model for Training

In [ ]:
# Train the Model
history_mobilenet = model_mobilenet.fit(
    train_set,
    epochs=10,
    steps_per_epoch=len(train_set.classes) // batch_size,
    validation_data=validation_set,
    callbacks=[early_stop, lr_scheduler, m_checkpoint],
    verbose=1,
)

### Save Model

In [ ]:
model_mobilenet.save("outputs/v1/mobilenet.h5")

---

# Model Performance

---

## Learning Curves 

In [ ]:
import plotly.express as px
import matplotlib.pyplot as plt

output_dir = "outputs/v1"

# Extract history from the training process
history_dict = history_mobilenet.history

# Convert history dictionary to a long format for Plotly
df_long = pd.DataFrame({
    'Epoch': range(1, len(history_dict['loss']) + 1),
    'loss': history_dict['loss'],
    'val_loss': history_dict['val_loss'],
    'accuracy': history_dict['accuracy'],
    'val_accuracy': history_dict['val_accuracy']
})

# Convert to long format
df_long = df_long.melt(id_vars=["Epoch"], value_vars=["loss", "val_loss", "accuracy", "val_accuracy"],
                       var_name="Metric", value_name="Value")

# Split into two DataFrames: one for loss, one for accuracy
df_loss = df_long[df_long["Metric"].isin(["loss", "val_loss"])]
df_acc = df_long[df_long["Metric"].isin(["accuracy", "val_accuracy"])]

# Interactive Loss Curve
fig_loss = px.line(
    df_loss,
    x="Epoch",
    y="Value",
    color="Metric",
    markers=True,
    title="Loss - MobileNetV2",
    labels={"Value": "Loss", "Epoch": "Epoch"},
    template="plotly_white",
)

# Interactive Accuracy Curve
fig_acc = px.line(
    df_acc,
    x="Epoch",
    y="Value",
    color="Metric",
    markers=True,
    title="Accuracy - MobileNetV2",
    labels={"Value": "Accuracy", "Epoch": "Epoch"},
    template="plotly_white",
)

# Save the figures as static PNGs
fig_loss.write_image(f"{output_dir}/model_training_losses.png", scale=2)
fig_acc.write_image(f"{output_dir}/model_training_acc.png", scale=2)

# Show interactive plots
fig_loss.show()
fig_acc.show()

- **Accuracy:** Training accuracy quickly reaches **~1.0**, while validation accuracy steadily improves with a small gap (~1%).  
- **Loss:** Both training and validation loss decrease consistently, showing stable learning with **no major overfitting**.  

---

# Model Evaluation

### Import Packages

In [ ]:
import sklearn
import sklearn.metrics as metrics
from sklearn.metrics import (
    classification_report,
    confusion_matrix,
    f1_score,
    accuracy_score,
)

### Load Saved Model

In [ ]:
from keras.models import load_model

# Load the model
model_mobilenet = load_model("outputs/v1/mobilenet.h5")

## Evaluation on Test Set

In [ ]:
evaluation = model_mobilenet.evaluate(test_set, batch_size=batch_size)
print("Model accuracy: {:.2f}%".format(evaluation[1] * 100))
print("Model Loss: ", evaluation[0])

## Set Accuracy Variables

In [ ]:
# Correctly obtain true labels
y_true = test_set.labels

# Obtain model predictions
preds = model_mobilenet.predict(test_set)
y_pred = np.argmax(preds, axis=1)

## Histograms

The histogram below shows the overall distribution of model confidence scores for predictions on the test set. To explore individual image confidence levels and analyze misclassifications, please refer to the interactive version in the Streamlit dashboard.

In [ ]:
import numpy as np
import plotly.graph_objects as go
import plotly.express as px
import psutil

output_dir = "outputs/v1"

# Get predicted probabilities
y_pred_probs = model_mobilenet.predict(validation_set)

# Create DataFrame for Plotly
df_probs = pd.DataFrame({"Healthy": y_pred_probs[:, 0], "Infected": y_pred_probs[:, 1]})

# Create histogram traces for both classes
fig = go.Figure()

fig.add_trace(
    go.Histogram(
        x=df_probs["Healthy"],
        nbinsx=20,
        opacity=0.6,
        name="Healthy",
        marker_color="green",
    )
)

fig.add_trace(
    go.Histogram(
        x=df_probs["Infected"],
        nbinsx=20,
        opacity=0.6,
        name="Infected",
        marker_color="blue",
    )
)

# Add threshold line at 0.5
fig.add_trace(
    go.Scatter(
        x=[0.5, 0.5],
        y=[
            0,
            max(np.histogram(y_pred_probs[:, 1], bins=20)[0]),
        ],  # Adjust y-axis dynamically
        mode="lines",
        name="Threshold = 0.5",
        line=dict(color="red", dash="dash"),
    )
)

# Update layout
fig.update_layout(
    title="Prediction Probability Histogram",
    xaxis_title="Prediction Probability",
    yaxis_title="Count",
    barmode="overlay",
    template="plotly_white",
)

# Save the figure as a static PNG
fig.write_image(f"{output_dir}/histogram_test.png", scale=2)

# Show the interactive plot
fig.show()


# Cleanup Kaleido processes after saving the image
def cleanup_kaleido():
    for proc in psutil.process_iter(["pid", "name"]):
        if "kaleido" in proc.info["name"].lower():
            os.kill(proc.info["pid"], 9)  # Force kill


cleanup_kaleido()  # Run cleanup to prevent zombie processes

The histogram shows a clear separation between **healthy** and **infected** predictions, with most probabilities concentrated near 0 and 1. The decision threshold at **0.5** is well-placed, ensuring confident classifications. Minimal overlap suggests strong model confidence in distinguishing between classes.

## Confusion Matrix

In [ ]:
import numpy as np
import os
import pandas as pd
import plotly.figure_factory as ff
from sklearn.metrics import confusion_matrix
import psutil

output_dir = "outputs/v1"

# Get Class Labels
label_map = list(test_set.class_indices.keys())

# Evaluate Model on Train and Test Sets
y_true_train = train_set.classes
y_pred_train = np.argmax(model_mobilenet.predict(train_set), axis=1)

y_true_test = test_set.classes
y_pred_test = np.argmax(model_mobilenet.predict(test_set), axis=1)

# Generate Confusion Matrices for Train and Test Sets
cm_train = confusion_matrix(y_true_train, y_pred_train)
cm_test = confusion_matrix(y_true_test, y_pred_test)

# Convert confusion matrices to DataFrames
df_cm_train = pd.DataFrame(cm_train, index=label_map, columns=label_map)
df_cm_test = pd.DataFrame(cm_test, index=label_map, columns=label_map)

# Function to create interactive Confusion Matrix
def create_confusion_matrix_figure(conf_matrix, title):
    fig = ff.create_annotated_heatmap(
        z=conf_matrix.values,
        x=conf_matrix.columns.tolist(),
        y=conf_matrix.index.tolist(),
        colorscale="Blues",
        showscale=True,
    )

    fig.update_layout(
        title=title,
        xaxis_title="Predicted Label",
        yaxis_title="True Label",
        template="plotly_white",
    )
    return fig

# Create interactive plots for both train and test sets
fig_train = create_confusion_matrix_figure(df_cm_train, "Confusion Matrix - Train Set")
fig_test = create_confusion_matrix_figure(df_cm_test, "Confusion Matrix - Test Set")

# Save Figures as Static PNGs
fig_train.write_image(os.path.join(output_dir, "confusion_matrix_train.png"), scale=2)
fig_test.write_image(os.path.join(output_dir, "confusion_matrix_test.png"), scale=2)

# Show interactive plots one after another
fig_train.show()
fig_test.show()

print(f"Confusion Matrices saved at: {output_dir}/confusion_matrix_train.png & {output_dir}/confusion_matrix_test.png")


# Cleanup Kaleido processes after saving the image
def cleanup_kaleido():
    for proc in psutil.process_iter(["pid", "name"]):
        if "kaleido" in proc.info["name"].lower():
            os.kill(proc.info["pid"], 9)  # Force kill


cleanup_kaleido()  # Run cleanup to prevent zombie processes

The model performs well on both the training and test sets, with a high number of correct classifications. The test set shows **zero false positives**, meaning no healthy samples were misclassified as infected. A small number of false negatives (7 cases) suggest slight room for improvement in detecting infections, but overall accuracy remains strong.

## Classification Reports

In [ ]:
from sklearn.metrics import classification_report
import os

output_dir = "outputs/v1"

# Generate classification reports as text
report_train = classification_report(
    y_true_train, y_pred_train, target_names=label_map, digits=3
)
report_test = classification_report(
    y_true_test, y_pred_test, target_names=label_map, digits=3
)

# Print Train Report
print("\n### Classification Report - Train Set ###\n")
print(report_train)

# Print Test Report
print("\n### Classification Report - Test Set ###\n")
print(report_test)

# Save reports as text files
with open(f"{output_dir}/classification_report_train.txt", "w") as f:
    f.write(report_train)

with open(f"{output_dir}/classification_report_test.txt", "w") as f:
    f.write(report_test)

print("\nReports saved to outputs/v1/")


The classification report indicates that the model performs well on the test set, with perfect precision, recall, and F1-scores (1.000) for both classes. However, the train set results are significantly lower, showing a possible issue with overfitting or insufficient training.

## ROC Curves

In [ ]:
import numpy as np
import plotly.graph_objects as go
from sklearn.metrics import roc_curve, auc
import os
import psutil

output_dir = "outputs/v1"

# Generate predictions (probabilities) for both train and test sets
y_probs_train = model_mobilenet.predict(train_set)
y_probs_test = model_mobilenet.predict(test_set)

# Compute ROC curve for train and test sets
fpr_train, tpr_train, _ = roc_curve(y_true_train, y_probs_train[:, 1])
fpr_test, tpr_test, _ = roc_curve(y_true_test, y_probs_test[:, 1])

# Compute AUC for train and test sets
auc_train = auc(fpr_train, tpr_train)
auc_test = auc(fpr_test, tpr_test)

# Create an interactive Plotly figure
fig = go.Figure()

# Add Train ROC Curve
fig.add_trace(
    go.Scatter(
        x=fpr_train,
        y=tpr_train,
        mode="lines",
        name=f"Train AUC = {auc_train:.2f}",
        line=dict(color="blue"),
    )
)

# Add Test ROC Curve
fig.add_trace(
    go.Scatter(
        x=fpr_test,
        y=tpr_test,
        mode="lines",
        name=f"Test AUC = {auc_test:.2f}",
        line=dict(color="green"),
    )
)

# Add Random Guess Line (Baseline)
fig.add_trace(
    go.Scatter(
        x=[0, 1],
        y=[0, 1],
        mode="lines",
        name="Random (AUC = 0.50)",
        line=dict(color="black", dash="dash"),
    )
)

# Update layout
fig.update_layout(
    title="ROC Curve - Train vs Test",
    xaxis_title="False Positive Rate",
    yaxis_title="True Positive Rate",
    template="plotly_white",
    legend=dict(x=0.7, y=0.2),
)

# Save the figure as a static PNG
roc_curve_path = os.path.join(output_dir, "roc_curve.png")
fig.write_image(roc_curve_path, scale=2)

# Show interactive plot
fig.show()

print(f"ROC Curve saved at: {roc_curve_path}")


# Cleanup Kaleido processes after saving the image
def cleanup_kaleido():
    for proc in psutil.process_iter(["pid", "name"]):
        if "kaleido" in proc.info["name"].lower():
            os.kill(proc.info["pid"], 9)  # Force kill


cleanup_kaleido()  # Run cleanup to prevent zombie processes

The test ROC curve shows **perfect AUC (1.00)**, indicating strong model performance with **no false positives**. However, the train AUC is low (**0.49**), suggesting a potential issue in training evaluation. Further investigation may be needed to confirm the validity of the training metrics.

## Save Evaluation Pickle

In [ ]:
joblib.dump(value=evaluation,
            filename=f"outputs/v1/evaluation.pkl")

## Business Goal Validation

In [ ]:
# Load evaluation results
evaluation_results = joblib.load("outputs/v1/evaluation.pkl")

# Extract final test accuracy
test_accuracy = evaluation_results[1]  # Since the second item is likely the accuracy

# Check requirement
accuracy_threshold = 0.90
if test_accuracy >= accuracy_threshold:
    print(f"Model meets the business requirement! (Accuracy: {test_accuracy:.2%})")
else:
    print(f"Model does NOT meet the requirement. (Accuracy: {test_accuracy:.2%})")

---

# Predict on New Images

---

## Load random image as PIL

In [ ]:
from tensorflow.keras.preprocessing import image

pointer = 66
label = labels[0] 
pil_image = image.load_img(test_path + '/'+ label + '/'+ os.listdir(test_path+'/' + label)[pointer],
                            target_size= image_shape, color_mode='rgb')

print(f" Image shape: {pil_image.size}, Image mode: {pil_image.mode}")
pil_image

### Convert Image to Array and Prepare for Model Input

In [ ]:
my_image = image.img_to_array(pil_image)
my_image = np.expand_dims(my_image, axis=0) / 255.0  
print(my_image.shape)

### Predict class probabilities

In [ ]:
pred_proba = model_mobilenet.predict(my_image)[0, 0]

target_map = {v: k for k, v in train_set.class_indices.items()}
pred_class = target_map[pred_proba < 0.5]

if pred_class == target_map[1]:
    pred_proba = 1 - pred_proba

print(f"{pred_class} {round(pred_proba*100, 2)}%")

## Complete Model Performance Report

In [ ]:
import time
import pandas as pd
from IPython.display import display

# Record training duration
end_time = time.time()
training_time = round(end_time - start_time, 2)

# Extract accuracy metrics from training history
train_accuracy = round(history_mobilenet.history["accuracy"][-1], 4)
val_accuracy = round(history_mobilenet.history["val_accuracy"][-1], 4)

# Directly get test accuracy
test_accuracy = (
    round(test_accuracy, 4) if isinstance(test_accuracy, (int, float)) else None
)

# Measure inference time
start_inf_time = time.time()
model.predict(my_image)  # Run prediction on a sample image
end_inf_time = time.time()

# Calculate inference time per image
inference_time = round((end_inf_time - start_inf_time), 4)

# Determine overfitting risk
overfitting_risk = "Yes" if train_accuracy - val_accuracy > 0.05 else "No"

# Store results in a structured format
results = {
    "Model": "MobileNetV2",
    "Training Accuracy": train_accuracy,
    "Validation Accuracy": val_accuracy,
    "Test Accuracy": test_accuracy,
    "Training Time (sec)": training_time,
    "Inference Time (sec/sample)": inference_time,
    "Batch Size": 16,
    "Overfitting Risk": overfitting_risk,
}

# Convert results into a DataFrame
df_results = pd.DataFrame([results])

# Save results to CSV (append mode)
csv_file = "training_results.csv"
df_results.to_csv(
    csv_file, mode="a", index=False, header=not pd.io.common.file_exists(csv_file)
)

# Display results as a table
display(df_results)

---

# Conclusion and Next Steps

---


We successfully developed a deep learning model for image classification using a structured, beginner-friendly approach.  

### **Key Achievements**
- **Explored Multiple Architectures** → Compared Sigmoid, Softmax, and MobileNetV2 across different trials.  
- **Comprehensive Evaluation** → Assessed models using accuracy, loss, confusion matrices, and ROC curves.  
- **Optimized for Generalization** → Selected the best-performing model with minimal overfitting.  
- **Deployment-Ready Model** → Finalized MobileNetV2 for real-world application.  

### **Next Steps: Model Deployment**
- **Web App Integration** → Implement a user-friendly Streamlit interface for real-time image classification.  
- **Model Deployment** → Load the trained model and deploy it on a cloud platform for accessibility.  

This deployment will enable efficient real-world usage, making automated classification accessible to users.  